# Gapfilling Observations with Synthetic data

## Create no-gaps datasets

In [ ]:
import folium
import numpy as np
import xarray as xr
import seaborn as sb
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import odc.geo.xr
from odc.geo.xr import assign_crs
import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

%matplotlib inline

In [ ]:
model_var='NDVI'
feat = 'CLIM'
base = '/g/data/os22/chad_tmp/AusENDVI/data/'

In [ ]:
desert = xr.open_dataset(f'{base}synthetic/{model_var}/{model_var}_{feat}_desert_synthetic_5km_monthly_1982_2022.nc')
nondesert = xr.open_dataset(f'{base}synthetic/{model_var}/{model_var}_{feat}_nondesert_synthetic_5km_monthly_1982_2022.nc')
yy = xr.where(np.isnan(desert), nondesert, desert)

obs = xr.open_dataset(f'{base}{model_var}_harmonization/LGBM/{model_var}_{feat}_LGBM_5km_monthly_1982_2022_wGaps.nc')

if model_var=='LST':
    yy = yy-273.15
    obs = obs-273.15

# obs = obs.sel(time=yy.time)

In [ ]:
# yy['LST'].sel(time='1994').plot.imshow(robust=True, cmap='BrBG', col='time', col_wrap=6);

In [ ]:
# obs['LST'].sel(time='1994').plot.imshow(robust=True, cmap='BrBG', col='time', col_wrap=6);

### Gapfill small gaps with linear interpolation

In [ ]:
# seperate into climatologies and anomalies
obs_monthly = obs.groupby('time.month').mean()
obs_anom = obs.groupby('time.month') - obs_monthly  

# fill linearly by one-step
obs_anom = obs_anom.interpolate_na(dim='time', method='linear', limit=1)

#recombine anomalies and climatology
obs_fill = obs_anom.groupby('time.month') + obs_monthly
obs_fill = assign_crs(obs_fill.drop('month'), crs='epsg:4326')

### Fill remaining data-gaps with synthetic data

In [ ]:
syn_fill = obs_fill.fillna(yy[model_var])

### Remove values less than zero

In [ ]:
syn_fill = syn_fill.where(syn_fill['NDVI']>=0)

yy = yy.where(yy>=0)

## Save to disk

In [ ]:
#export here for the carbon flux modelling
syn_fill.to_netcdf(base+'5km/'+model_var+'_5km_monthly_1982_2022.nc')

In [ ]:
#export here for publication
syn_fill.to_netcdf(f'{base}{model_var}_harmonization/LGBM/{model_var}_{feat}_LGBM_5km_monthly_1982_2022_noGaps.nc')

yy.to_netcdf(f'{base}/synthetic/NDVI/NDVI_CLIM_synthetic_5km_monthly_1982_2022.nc')

## Before and after gapfill

In [ ]:
with plt.style.context('ggplot'):
    fig,ax = plt.subplots(1,1, figsize=(12,4), sharey=True)
    obs['NDVI'].mean(['latitude', 'longitude']).rolling(time=3).mean().plot(ax=ax, label='MCD43A4 & AusE-clim', c='red')
    syn_fill['NDVI'].mean(['latitude', 'longitude']).rolling(time=3).mean().plot(ax=ax, label='MCD43A4 & AusE-clim gapfilled', c='black')
    ax.set_ylabel('NDVI')
    ax.set_xlabel(None)
    ax.legend()
    plt.title(None);
    fig.savefig('/g/data/os22/chad_tmp/AusENDVI/results/figs/gapfill_before_after.png',
            bbox_inches='tight', dpi=300)

## Animations

In [ ]:
import xarray as xr
import sys
sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/')
from dea_tools.plotting import xr_animation
from IPython.display import Image
import matplotlib.pyplot as plt

In [ ]:
model_var = 'AusENDVI_synthetic'
ds = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/results/publication/AusENDVI_synthetic_1982_2022_0.2.0.nc')

In [ ]:
# # Produce time series animation
# path = '/g/data/os22/chad_tmp/AusENDVI/results/gifs/'+model_var+'_observed.gif'
# path = '/g/data/os22/chad_tmp/AusENDVI/results/gifs/'+model_var+'_synthetic_LGBM.gif'
path = '/g/data/os22/chad_tmp/AusENDVI/results/gifs/'+model_var+'_gapfilled_LGBM.gif'


cmap='gist_earth_r'
vmin=0.1
vmax=0.8

# if model_var=='LST':
#     cmap='inferno'
#     vmin=5
#     vmax=45

xr_animation(
             ds=ds,#obs.rolling(time=2, min_periods=1).mean(), 
             # ds=yy,#.rolling(time=2, min_periods=1).mean(), 
             # syn_fill.rolling(time=2, min_periods=1).mean(), 
             output_path=path,
             bands=[model_var],  
             show_date='%B-%Y',
             show_text=model_var,
             interval=150,
             imshow_kwargs={'cmap': cmap, 'vmin': vmin, 'vmax': vmax},
             colorbar_kwargs={'colors': 'black'},
             # width_pixels=300,
             annotation_kwargs= {'color':'black', 'animated': True}
    ) 

# Plot animated gif
plt.close()
Image(filename=path)